# Heat Index Calculator
This script calculates the heat index based on user input of temperature and dewpoint (DP).
From the temperature and dewpoint, the relative humidity is calculated. Then temperature and DP are used to calculate relative humidity (RH). Temperature and RH are the variables in the heat index equation
- source: NWS WPC "The Heat Index Equation": https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml
- The Heat Index Equation
    - HI = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
        - where T is temperature in degrees F and RH is relative humidity in percent.  
    - HI is the heat index expressed as an apparent temperature in degrees F.
- Adjustments to the Heat Index Equation
    - If the RH is less than 13% and the temperature is between 80 and 112 degrees F, then the following adjustment is _subtracted_ from HI: 
        - ADJUSTMENT = [(13-RH)/4]*SQRT{[17-ABS(T-95.)]/17}
    - If the RH is greater than 85% and the temperature is between 80 and 87 degrees F, then the following adjustment is _added_ to HI: 
        - ADJUSTMENT = [(RH-85)/10] * [(87-T)/5]
- The Rothfusz regression is not appropriate when conditions of temperature and humidity warrant a heat index value below about 80 degrees F. In those cases, a simpler formula is applied:
    - HI = 0.5 * {T + 61.0 + [(T-68.0)*1.2] + (RH*0.094)} 
- In practice, the simple formula is computed first and the result averaged with the temperature. If this heat index value is 80 degrees F or higher, the full regression equation along with any adjustment as described above is applied. 
- Calculate RH from DP
    - source: University of Miami (https://bmcnoldy.earth.miami.edu/Humidity.html)
    - RH: =100*(EXP((17.625*TD)/(243.04+TD))/EXP((17.625*T)/(243.04+T))) 
        - T = temp in _°Celsius_ 
        - TD = dew point temp (DP) in _°Celsius_ 

## Plan of Action
- Use main branch to dial in the calculation
    1) request user input of temp and DP
    2) request user input of whether temp and DP are in C or F
    3) _if_ T and DP not already in C, convert to C
    4) calculate RH (in C) from T and DP (in C)
    5) calculate HI (in F) via the simple HI formula
    6) _if_ HI > 80F, calculate full regression equation
    7) _if_ RH > 85% _and_ 80F < T < 87F, add appropriate adjustment to HI
    8) _if_ RH < 13% _and_ 80F < T < 112F, subtract appropriate adjustment to HI
    9) print calculated HI 
- Branch the functioning calculator and add additional comments as output, e.g.:
    - "Humidity in the air makes it feel X degrees warmer than the air temperature"
- Merge the calculator with comment capability back onto main

In [28]:
# Import necessary modules

import math

In [72]:
# HEAT INDEX CALCULATOR
# User inputs = temperature and dew point in either degrees C or degrees F

# Request user inputs
temp = input("Enter the air temperature as a whole number, in degrees C or degrees F) \n")
temp = int(temp)
dewpt = input("Enter the dew point temperature as a whole number, in degrees C or degrees F) \n")
dewpt = int(dewpt)
t_scale = input("Are the temperature and dew point in degrees Celcius or degrees Fahrenheit? Enter 'C' for Celcius or 'F' for Fahrenheit) \n")

In [92]:
# reset temp and dewpt without needing inputs
# DELETE IN FINAL SCRIPT

temp = 82
dewpt = 81
t_scale = 'F'

In [93]:
# Print out the variables the user entered (confirmation)
print ("You entered:")
print("temp = ", temp)
print("dewpt = ", dewpt)
print("temperature scale = ", t_scale)

You entered:
temp =  82
dewpt =  81
temperature scale =  F


In [94]:
# The HI formulas require temp and dewpt in Fahrenheit, whereas the RH forumula requires temp and dewpt in Celcius, so save the temp in both F and C

# Possible error: this code assumes that the user correctly entered the temp scale as either F or C
# I'm not including error handling for any other possible value of t_scale 

# If temp and dewpt were entered in Fahrenheit, convert to Celcius
if t_scale == 'F':
    tempF = temp
    dewptF = dewpt

    tempC = round((tempF - 32)*(5/9),1)
    dewptC = round((dewptF - 32)*(5/9),1)

# If temp and dewpt were entered in Celcius, convert to Fahrenheit
else:
    tempC = temp
    dewptC = dewpt

    tempF =(tempC*(9/5)) + 32
    dewptF = (dewptC*(9/5)) + 32

In [95]:
# DELETE IN FINAL SCRIPT - checking calculations
print("user-entered temp = ", temp)
print("user-entered dewpt = ", dewpt)
print("temp in F = ", tempF)
print("dewpt in F = ", dewptF)
print("temp in C = ", tempC)
print("dewpt in C = ", dewptC)


user-entered temp =  82
user-entered dewpt =  81
temp in F =  82
dewpt in F =  81
temp in C =  27.8
dewpt in C =  27.2


In [96]:
# Calculate RH using temp and dewpt (in C)

rh =round(100*(math.exp((17.625*dewptC)/(243.04+dewptC))/math.exp((17.625*tempC)/(243.04+tempC))),2) 

In [97]:
print("At this temperature and dewpoint, the relative humidity is ", rh, "%")

At this temperature and dewpoint, the relative humidity is  96.55 %


In [98]:
# Calculate HI using simple forumula
# If this formula returns an HI > 80F, we need to use the full regression

# Units:
# temp and dewpt must be in degrees F
# rh must be in percent

heat_index_simp = round(0.5 * (tempF + 61.0 + ((tempF-68.0)*1.2) + (rh*0.094)),0)
print("simple HI = ", heat_index_simp)

# If the simple HI formula returns a value less than 80 degrees, we're done:
if heat_index_simp < 80:
    heat_index = heat_index_simp

else: 
    heat_index_reg = round((-42.379) + (2.04901523*tempF) + (10.14333127*rh) 
                       - (.22475541*tempF*rh) - (.00683783*tempF*tempF) 
                       - (.05481717*rh*rh) + (.00122874*tempF*tempF*rh) 
                       + (.00085282*tempF*rh*rh) - (.00000199*tempF*tempF*rh*rh),0)
    print ("regression HI without adjustments = ", heat_index_reg) 
    
    # apply any applicable adjustments
    if (rh < 13) and ((tempF > 80) and (tempF < 112)):
        print ("entering if statement for low RH")
        adj1 = ((13-rh)/4)*math.sqrt((17-abs(tempF-95))/17)

        heat_index_reg = round(heat_index_reg - adj1,0)

    if (rh > 85) and ((tempF > 80) and (tempF < 87)):
        print ("entering if statement for swamp")
        adj2 = ((rh-85)/10) * ((87-tempF)/5)

        heat_index_reg = round(heat_index_reg + adj2,0)
    
    heat_index = heat_index_reg

print("The heat index under these conditions is ", heat_index, "degrees Fahrenheit")

simple HI =  84.0
regression HI without adjustments =  93.0
entering if statement for swamp
The heat index under these conditions is  94.0 degrees Fahrenheit


In [99]:
# New feature:
# specify by how much the humidity raises the apparent temperature - i.e. how much higher is the HI than the air temp

t_diff = round(heat_index - tempF,0)
print ("The humidity in the air makes it feel", t_diff, "degrees warmer than the air temperature.")

The humidity in the air makes it feel 12.0 degrees warmer than the air temperature.
